In [10]:
from DataLoader import importar_ventas,leer_csv,agregar_fourier,preparar_test,generar_lag_features
from DataLoader import generar_rolling_features,crear_archivo_kaggle
import numpy as np
from sklearn.model_selection import GridSearchCV,PredefinedSplit, RandomizedSearchCV
from xgboost import XGBRegressor
import pandas as pd

In [8]:
def predecir(separador):
    return 

In [6]:
ventas=importar_ventas()
ventas
agregar_fourier(ventas,2)
#ventas=generar_rolling_features(ventas,"demand",[1,2],["subgroup_cod","store_cod"])
ventas=generar_rolling_features(ventas,"mean_price",[1,2],["subgroup_cod","store_cod"])
ventas=generar_rolling_features(ventas,"total_sales",[1,2],["subgroup_cod","store_cod"])


In [11]:
for i in range(1,8): #agregamos lag a 7 dias por que vamos a predecir 7 dias
	generar_lag_features(ventas,"total_sales",i)

In [14]:
ventas["total_sales"]=ventas["total_sales"].fillna(0)

In [4]:
print(len(ventas["group"].unique()))
for group in ventas["group"].unique():
    print(len(ventas[ventas["group"]==group])) #tambien podria ser

23
323148
210201
212403
188989
182576
173558
209727
102800
366582
189105
204801
184253
258450
174080
184564
279056
271314
208328
344324
161916
194389
223177
359052


In [61]:
#model=XGBRegressor()
separador="group"
bins=ventas[separador].unique()
TARGET=["demand"]
FEATURES=[#'mean_price', #'std_price', 
          'max_price', 'min_price',
        'cluster', #'store_cod', 
        # 'subgroup_cod', #'day',
       #'month', 
        'year', 'anual_sin_1',
    'anual_cos_1', 'mensual_sin_1',
       #'mensual_cos_1', 
        #'anual_sin_2', 'anual_cos_2', 
    'mensual_sin_2', #'demand_rolling_mean_7d',
       'demand_rolling_sum_7d', 
    #'demand_rolling_mean_28d',
       #'demand_rolling_sum_28d'
       #'mensual_cos_2',
        # 'mean_price_rolling_mean_7d',
       #'mean_price_rolling_sum_7d', 
    #'mean_price_rolling_mean_28d',
       #'mean_price_rolling_sum_28d'   
]
evals_result={}

# Diccionario para guardar resultados por bucket
resultados_por_bucket = {}

# Grilla de hiperparámetros
param_grid = {
    'n_estimators': [200,350,500],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1],
    #'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
    #'reg_alpha': [0, 5],
    #'reg_lambda': [1, 5]
    "objective":["reg:squaredlogerror","reg:squarederror"]
}

for bucket in bins:
    data_bucket = ventas[ventas[separador] == bucket]
    
    # Crear test: últimas 7 filas por grupo
    test = data_bucket.groupby(['store_cod', 'subgroup_cod']).tail(7)
    train = data_bucket.drop(test.index)
    data_bucket.loc[data_bucket.index,"ps"]=-1
    data_bucket.loc[test.index,"ps"]=0

    test_fold=data_bucket["ps"]
    # Creamos el split
    ps = PredefinedSplit(test_fold)

    #definimos el modelo
    
    reg = XGBRegressor(
    #reducir el numero de columnas aumenta la importancia de std_price
    )

    grid_search = GridSearchCV(
        estimator=reg,
        param_grid=param_grid,
        scoring='neg_root_mean_squared_error',
        cv=ps,
        verbose=2,
        n_jobs=-1
    )
    
    grid_search.fit(data_bucket[FEATURES], data_bucket[TARGET])

    # Guardar resultados
    resultados_por_bucket[bucket] = {
        'best_params': grid_search.best_params_,
        'best_score': -grid_search.best_score_  # lo paso a positivo
    }
    
    print("Mejores parámetros:", grid_search.best_params_)
    print("Mejor RMSE en validación:", -grid_search.best_score_)    

C:\Users\emili\AppData\Local\Temp\ipykernel_11564\1824466024.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 48 candidates, totalling 48 fits
Mejores parámetros: {'colsample_bytree': 1, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 500, 'objective': 'reg:squarederror'}
Mejor RMSE en validación: 2.311519698926529


C:\Users\emili\AppData\Local\Temp\ipykernel_11564\1824466024.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 48 candidates, totalling 48 fits
Mejores parámetros: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 200, 'objective': 'reg:squarederror'}
Mejor RMSE en validación: 2.0809230770998046


C:\Users\emili\AppData\Local\Temp\ipykernel_11564\1824466024.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


...

Fitting 1 folds for each of 48 candidates, totalling 48 fits
Mejores parámetros: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 500, 'objective': 'reg:squarederror'}
Mejor RMSE en validación: 2.4271779823607127


C:\Users\emili\AppData\Local\Temp\ipykernel_11564\1824466024.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 48 candidates, totalling 48 fits
Mejores parámetros: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 500, 'objective': 'reg:squaredlogerror'}
Mejor RMSE en validación: 2.5717356758920578


C:\Users\emili\AppData\Local\Temp\ipykernel_11564\1824466024.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 48 candidates, totalling 48 fits
Mejores parámetros: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 200, 'objective': 'reg:squaredlogerror'}
Mejor RMSE en validación: 2.1824601949967866


In [65]:
resultados_por_bucket
acum=0
for key in resultados_por_bucket.keys():
    
    acum+=resultados_por_bucket[key]['best_score']
    print(key+ " rmse: "+str(resultados_por_bucket[key]['best_score']))
print(acum/len(resultados_por_bucket.keys()))

Men rmse: 2.311519698926529
Fiction rmse: 2.0809230770998046
Audio rmse: 2.0183582906692012
Computing rmse: 2.4648140231563977
Games rmse: 2.1968518354016546
Outdoor rmse: 2.411669001644258
Non-Fiction rmse: 2.0724137803376883
Team Sports rmse: 2.272388833569273
Educational rmse: 2.270911223251704
Skincare rmse: 2.0774338692251093
Cookware rmse: 2.0230864178977397
Appliances rmse: 2.0635550250064805
Children rmse: 2.1366228068923583
Fitness rmse: 2.1775094235977126
Furniture rmse: 1.9235999872193807
Pantry rmse: 2.1665689931245584
Beverages rmse: 2.1385269090448094
Mobile rmse: 1.8185567610476894
Snacks rmse: 2.5060655733276005
Action rmse: 2.1976055304499145
Haircare rmse: 2.4271779823607127
Makeup rmse: 2.5717356758920578
Women rmse: 2.1824601949967866
2.196102387571279


In [ ]:
from inline_sql import sql, sql_val

taka=sql^"""SELECT DISTINCT category, "group" FROM ventas order by category"""
taka["dict"]=taka["group"].map(resultados_por_bucket)
taka["rmse"]=0
taka["params"]={}
for i in range(len(taka)):
    fila=taka["dict"].iloc[i]
    taka["rmse"].iloc[i]=fila["best_score"]
    taka["params"].iloc[i]=fila["best_params"]
taka.to_csv("prueba_parametro_por_grupo.csv")

In [87]:
taka.to_csv("prueba_parametro_por_grupo.csv")

In [16]:
#randomized_search
#model=XGBRegressor()
separador="category"
bins=ventas[separador].unique()
TARGET=["total_sales"]
FEATURES=[
 #  'mean_price', 
   # 'cluster', 
   # 'store_cod', 
    'subgroup_cod',
   'total_sales_lag_1',
    #'total_sales_lag_2', 
   # 'total_sales_lag_3', #'total_sales_lag_4',
    #'total_sales_lag_5', 'total_sales_lag_6', 'total_sales_lag_7', 
    #'day', 'month', 
    'year',
   'anual_sin_1', 
    'anual_cos_1',
  'mensual_sin_1', 
  #  'mensual_cos_1',
 #  'anual_sin_2', 
 #   'anual_cos_2',
   'mensual_sin_2', 
 #   'mensual_cos_2',
    #"std_price",
   # "max_price", 
   #"min_price",
    'mean_price_rolling_mean_7d',
   # 'mean_price_rolling_sum_7d',
  #   'mean_price_rolling_mean_14d',
    'mean_price_rolling_sum_14d', 
   # 'mean_price_rolling_mean_21d',#  'mean_price_rolling_sum_21d', 'mean_price_rolling_mean_28d',
    #  'mean_price_rolling_sum_28d', 
    #'demand_rolling_mean_7d',
    #  'demand_rolling_sum_7d', 
    #'demand_rolling_mean_14d',
     'total_sales_rolling_mean_7d',
       'total_sales_rolling_sum_7d', 
  #  'total_sales_rolling_mean_14d',
   #    'total_sales_rolling_sum_14d',
    #'std_discount', 'max_discount', 
    #'min_discount', 
    #'std_diff_factory',
    #"mean_discount",
    #"mean_diff_factory",
   # 'max_diff_factory', 
   # 'min_diff_factory',
    #"size"
]
evals_result={}

# Diccionario para guardar resultados por bucket
resultados_por_bucket_rnd = {}

# Grilla de hiperparámetros
param_grid = {
    'n_estimators': [200,350,500],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1],
    #'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
    'reg_alpha': [0, 5,10,20],
    'reg_lambda': [1, 5],
    "objective":["reg:squaredlogerror","reg:squarederror"]
}

for bucket in bins:
    data_bucket = ventas[ventas[separador] == bucket]
    
    # Crear test: últimas 7 filas por grupo
    test = data_bucket.groupby(['store_cod', 'subgroup_cod']).tail(7)
    train = data_bucket.drop(test.index)
    data_bucket.loc[data_bucket.index,"ps"]=-1
    data_bucket.loc[test.index,"ps"]=0

    X_test=test[FEATURES]
    y_test=test[TARGET]

    test_fold=data_bucket["ps"]
    # Creamos el split
    ps = PredefinedSplit(test_fold)

    fit_params = {
    "eval_set": [(X_test, y_test)],
    "verbose": False
    }

    #definimos el modelo
    n_iter_search=10
    reg = XGBRegressor(
        early_stopping_rounds= 30,
        eval_metric= "rmse",
    #reducir el numero de columnas aumenta la importancia de std_price
    )


    rand_search = RandomizedSearchCV(
    estimator=reg,
    param_distributions=param_grid,
    n_iter=n_iter_search,
    scoring='neg_root_mean_squared_error',
    cv=ps,  # tu PredefinedSplit definido antes
    verbose=2,
    n_jobs=-1,
    random_state=42
)
    rand_search.fit(data_bucket[FEATURES], data_bucket[TARGET],**fit_params)

    # Guardar resultados
    resultados_por_bucket_rnd[bucket] = {
        'best_params': rand_search.best_params_,
        'best_score': -rand_search.best_score_  # lo paso a positivo
    }
    
    print("Mejores parámetros:", rand_search.best_params_)
    print("Mejor RMSE en validación:", -rand_search.best_score_)    



C:\Users\emili\AppData\Local\Temp\ipykernel_15064\3659359380.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 10 candidates, totalling 10 fits


C:\Users\emili\AppData\Roaming\Python\Python312\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Mejores parámetros: {'reg_lambda': 1, 'reg_alpha': 20, 'objective': 'reg:squarederror', 'n_estimators': 500, 'max_depth': 4, 'learning_rate': 0.05, 'colsample_bytree': 1}
Mejor RMSE en validación: 143.1101768313421


C:\Users\emili\AppData\Local\Temp\ipykernel_15064\3659359380.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


...

Fitting 1 folds for each of 10 candidates, totalling 10 fits
Mejores parámetros: {'reg_lambda': 1, 'reg_alpha': 20, 'objective': 'reg:squarederror', 'n_estimators': 500, 'max_depth': 4, 'learning_rate': 0.05, 'colsample_bytree': 1}
Mejor RMSE en validación: 68.11026895479833


C:\Users\emili\AppData\Local\Temp\ipykernel_15064\3659359380.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 10 candidates, totalling 10 fits
Mejores parámetros: {'reg_lambda': 1, 'reg_alpha': 10, 'objective': 'reg:squarederror', 'n_estimators': 500, 'max_depth': 4, 'learning_rate': 0.1, 'colsample_bytree': 1}
Mejor RMSE en validación: 217.3305124426368


C:\Users\emili\AppData\Local\Temp\ipykernel_15064\3659359380.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 10 candidates, totalling 10 fits
Mejores parámetros: {'reg_lambda': 1, 'reg_alpha': 10, 'objective': 'reg:squarederror', 'n_estimators': 350, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 1}
Mejor RMSE en validación: 257.67028153133117


In [ ]:
#clothing Mejores parámetros: {'objective': 'reg:squarederror', 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 1}

In [ ]:
from inline_sql import sql, sql_val

taka=sql^"""SELECT DISTINCT category FROM ventas order by category"""
taka["dict"]=taka["category"].map(resultados_por_bucket_rnd)
taka["rmse"]=0
taka["params"]={}
for i in range(len(taka)):
     fila=taka["dict"].iloc[i]
     taka["rmse"].iloc[i]=fila["best_score"]
     taka["params"].iloc[i]=fila["best_params"]
taka.to_csv("prueba_parametro_por_categoria.csv")
#resultados_por_bucket
taka

In [18]:
taka[["category","rmse"]]
#Beauty pesimo
#books ta weno
#clothing same
#Electronics	
#Groceries	
#Home & Kitchen	
#Sports	
#Toys

,category,rmse
0,Beauty,114.991494
1,Books,68.110269
2,Clothing,143.110177
3,Electronics,781.866588
4,Groceries,32.465521
5,Home & Kitchen,217.330512
6,Sports,257.670282
7,Toys,85.908446


In [50]:
import numpy as np
from sklearn.metrics import mean_squared_error

separador="category"
bins=ventas[separador].unique()
TARGET=["total_sales"]
FEATURES=[
 #  'mean_price', 
   # 'cluster', 
   # 'store_cod', 
    'subgroup_cod',
   'total_sales_lag_1',
    #'total_sales_lag_2', 
   # 'total_sales_lag_3', #'total_sales_lag_4',
    #'total_sales_lag_5', 'total_sales_lag_6', 'total_sales_lag_7', 
    #'day', 'month', 
    'year',
   'anual_sin_1', 
    'anual_cos_1',
  'mensual_sin_1', 
  #  'mensual_cos_1',
 #  'anual_sin_2', 
 #   'anual_cos_2',
   'mensual_sin_2', 
 #   'mensual_cos_2',
    #"std_price",
   # "max_price", 
   #"min_price",
    'mean_price_rolling_mean_7d',
   # 'mean_price_rolling_sum_7d',
  #   'mean_price_rolling_mean_14d',
    'mean_price_rolling_sum_14d', 
    #'demand_rolling_mean_7d',
    #  'demand_rolling_sum_7d', 
    #'demand_rolling_mean_14d',
     'total_sales_rolling_mean_7d',
       'total_sales_rolling_sum_7d', 
    'total_sales_rolling_mean_14d',
       'total_sales_rolling_sum_14d',
    #'std_discount', 'max_discount', 
    #'min_discount', 
    #'std_diff_factory',
    #"mean_discount",
    #"mean_diff_factory",
   # 'max_diff_factory', 
   # 'min_diff_factory',
    #"size"
]
evals_result={}

# Diccionario para guardar resultados por bucket
resultados_por_bucket_rnd = {}

for bucket in bins:
    data_bucket = ventas[ventas[separador] == bucket]
    
    # Crear test: últimas 7 filas por grupo
    test = data_bucket.groupby(['store_cod', 'subgroup_cod']).tail(1)
    train = data_bucket.drop(test.index)
    data_bucket.loc[data_bucket.index,"ps"]=-1
    data_bucket.loc[test.index,"ps"]=0
    
    X_train=data_bucket[FEATURES]
    y_train=np.log1p(data_bucket[TARGET])
    

    X_test=test[FEATURES]
    y_test=np.log1p(test[TARGET])

    # Creamos el split
    ps = PredefinedSplit(test_fold)
    def rmsle_obj(y_pred, y_true):
        y_true = y_true.get_label()
        grad = (np.log1p(y_pred) - np.log1p(y_true)) / (y_pred + 1)
        hess = (1 - (np.log1p(y_pred) - np.log1p(y_true))) / ((y_pred + 1) ** 2)
        return grad, hess

    #definimos el modelo
   
    reg = XGBRegressor(
     n_estimators=200,           # alto para usar early stopping
    learning_rate=0.05,          # más bajo para no memorizar rápido
    max_depth=6,                 # menos profundidad => menos overfit
    #min_child_weight=5,          # evita que splits se hagan con poca data
    #subsample=0.8,                # usa solo parte de las filas en cada árbol
    colsample_bytree=0.8,         # usa solo parte de las columnas en cada árbol
    reg_lambda=20,                 # L2 regularization
    reg_alpha=0,                  # L1 regularization
    random_state=42,
    n_jobs=-1,
    early_stopping_rounds=50,
    objective=rmsle_obj,
    eval_metric="rmse"
    )
    reg.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=50,
    )
    print(bucket)
    print(pd.DataFrame({
    "feature": FEATURES,
    "importance": reg.feature_importances_
}).sort_values("importance", ascending=False))
    rmse_original = np.sqrt(mean_squared_error(np.expm1(y_test),np.expm1(reg.predict(X_test))))
    print(rmse_original)

C:\Users\emili\AppData\Local\Temp\ipykernel_15064\4026203582.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


[0]	validation_0-rmse:0.69858
[50]	validation_0-rmse:0.67310
[100]	validation_0-rmse:0.66694
[150]	validation_0-rmse:0.66492
[199]	validation_0-rmse:0.66443
Clothing
                         feature  importance
11  total_sales_rolling_mean_14d    0.261860
1              total_sales_lag_1    0.258523
2                           year    0.103687
3                    anual_sin_1    0.085650
12   total_sales_rolling_sum_14d    0.076523
6                  mensual_sin_2    0.055819
7     mean_price_rolling_mean_7d    0.039242
4                    anual_cos_1    0.035814
5                  mensual_sin_1    0.024498
9    total_sales_rolling_mean_7d    0.022097
0                   subgroup_cod    0.018932
10    total_sales_rolling_sum_7d    0.012785
8     mean_price_rolling_sum_14d    0.004569
140.89666047427875


C:\Users\emili\AppData\Local\Temp\ipykernel_15064\4026203582.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


[0]	validation_0-rmse:0.62673
[50]	validation_0-rmse:0.61690
[100]	validation_0-rmse:0.61465
[150]	validation_0-rmse:0.61360
[199]	validation_0-rmse:0.61240
Beauty
                         feature  importance
1              total_sales_lag_1    0.208632
11  total_sales_rolling_mean_14d    0.195726
10    total_sales_rolling_sum_7d    0.154750
2                           year    0.077240
3                    anual_sin_1    0.067484
6                  mensual_sin_2    0.063144
12   total_sales_rolling_sum_14d    0.061189
9    total_sales_rolling_mean_7d    0.043166
4                    anual_cos_1    0.039688
5                  mensual_sin_1    0.038507
7     mean_price_rolling_mean_7d    0.032309
0                   subgroup_cod    0.013860
8     mean_price_rolling_sum_14d    0.004305
105.42789768248296


C:\Users\emili\AppData\Local\Temp\ipykernel_15064\4026203582.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


...

[0]	validation_0-rmse:0.61785
[50]	validation_0-rmse:0.60923
[100]	validation_0-rmse:0.60593
[150]	validation_0-rmse:0.60281
[199]	validation_0-rmse:0.60020
Books
                         feature  importance
1              total_sales_lag_1    0.293830
11  total_sales_rolling_mean_14d    0.189312
3                    anual_sin_1    0.111227
6                  mensual_sin_2    0.093902
2                           year    0.070419
12   total_sales_rolling_sum_14d    0.060418
7     mean_price_rolling_mean_7d    0.047343
4                    anual_cos_1    0.040525
9    total_sales_rolling_mean_7d    0.031235
5                  mensual_sin_1    0.024718
0                   subgroup_cod    0.019159
10    total_sales_rolling_sum_7d    0.010315
8     mean_price_rolling_sum_14d    0.007597
70.1731399087308


C:\Users\emili\AppData\Local\Temp\ipykernel_15064\4026203582.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


[0]	validation_0-rmse:0.63221
[50]	validation_0-rmse:0.62576
[100]	validation_0-rmse:0.62400
[150]	validation_0-rmse:0.62302
[199]	validation_0-rmse:0.62203
Home & Kitchen
                         feature  importance
1              total_sales_lag_1    0.260797
10    total_sales_rolling_sum_7d    0.198558
11  total_sales_rolling_mean_14d    0.100485
12   total_sales_rolling_sum_14d    0.097813
3                    anual_sin_1    0.070209
6                  mensual_sin_2    0.062618
4                    anual_cos_1    0.042671
2                           year    0.042311
7     mean_price_rolling_mean_7d    0.040748
9    total_sales_rolling_mean_7d    0.038025
5                  mensual_sin_1    0.027596
0                   subgroup_cod    0.014166
8     mean_price_rolling_sum_14d    0.004003
217.47416481373014


C:\Users\emili\AppData\Local\Temp\ipykernel_15064\4026203582.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


[0]	validation_0-rmse:0.61795
[50]	validation_0-rmse:0.61651
[100]	validation_0-rmse:0.61500
[150]	validation_0-rmse:0.61184
[199]	validation_0-rmse:0.61081
Sports
                         feature  importance
1              total_sales_lag_1    0.345226
11  total_sales_rolling_mean_14d    0.142283
6                  mensual_sin_2    0.109159
2                           year    0.099978
3                    anual_sin_1    0.085068
7     mean_price_rolling_mean_7d    0.051068
9    total_sales_rolling_mean_7d    0.043004
4                    anual_cos_1    0.036164
12   total_sales_rolling_sum_14d    0.031183
5                  mensual_sin_1    0.030592
0                   subgroup_cod    0.013047
10    total_sales_rolling_sum_7d    0.008464
8     mean_price_rolling_sum_14d    0.004764
241.82119567972367


In [25]:
bins=ventas[separador].unique()
bins

array(['Clothing', 'Beauty', 'Electronics', 'Toys', 'Groceries', 'Books',
       'Home & Kitchen', 'Sports'], dtype=object)

In [35]:
ventas[ventas["category"]=="Electronics"]["total_sales"].describe()

count    609720.000000
mean       1411.419677
std        1008.757199
min           0.000000
25%         723.097500
50%        1169.335000
75%        1805.082500
max       17802.700000
Name: total_sales, dtype: float64

In [42]:
import pandas as pd
import numpy as np


for bucket in bins:
    data_bucket = ventas[ventas[separador] == bucket]
    # Ejemplo de serie
    s = data_bucket["total_sales"]
    # Calcular percentiles específicos
    percentiles = [25,50,75,90, 95, 98]
    valores = np.percentile(s, percentiles)
    print(bucket)
    # Crear un DataFrame con percentiles y sus índices originales
    resultado = pd.DataFrame({
        'percentil': percentiles,
        'valor': valores,
        'max':s.max()
        #'indice': [s.index[s == v][0] for v in valores]
    })
    
    print(resultado)
    print()
        

Clothing
   percentil     valor      max
0         25  130.6500  3174.04
1         50  213.9800  3174.04
2         75  333.7400  3174.04
3         90  490.6800  3174.04
4         95  607.6655  3174.04
5         98  764.4600  3174.04

Beauty
   percentil    valor      max
0         25   99.870  3397.14
1         50  160.900  3397.14
2         75  247.660  3397.14
3         90  366.570  3397.14
4         95  464.695  3397.14
5         98  610.606  3397.14

Electronics
   percentil      valor      max
0         25   723.0975  17802.7
1         50  1169.3350  17802.7
2         75  1805.0825  17802.7
3         90  2639.8010  17802.7
4         95  3295.0505  17802.7
5         98  4214.7462  17802.7

Toys
   percentil    valor      max
0         25   76.830  1573.27
1         50  121.470  1573.27
2         75  182.050  1573.27
3         90  259.050  1573.27
4         95  318.230  1573.27
5         98  397.025  1573.27

Groceries
   percentil   valor     max
0         25   30.40  947.49
1     

In [44]:
import pandas as pd
import numpy as np


for bucket in bins:
    data_bucket = ventas[ventas[separador] == bucket]
    # Ejemplo de serie
    s = np.log1p(data_bucket["total_sales"])
    # Calcular percentiles específicos
    percentiles = [25,50,75,90, 95, 98]
    valores = np.percentile(s, percentiles)
    print(bucket)
    # Crear un DataFrame con percentiles y sus índices originales
    resultado = pd.DataFrame({
        'percentil': percentiles,
        'valor': valores,
        'max':s.max()
        #'indice': [s.index[s == v][0] for v in valores]
    })
    
    print(resultado)
    print()
        

Clothing
   percentil     valor       max
0         25  4.880147  8.063076
1         50  5.370545  8.063076
2         75  5.813354  8.063076
3         90  6.197828  8.063076
4         95  6.411269  8.063076
5         98  6.640477  8.063076

Beauty
   percentil     valor       max
0         25  4.613833  8.130984
1         50  5.086979  8.130984
2         75  5.516087  8.130984
3         90  5.906914  8.130984
4         95  6.143531  8.130984
5         98  6.416088  8.130984

Electronics
   percentil     valor       max
0         25  6.584926  9.787162
1         50  7.065045  9.787162
2         75  7.498915  9.787162
3         90  7.878838  9.787162
4         95  8.100480  9.787162
5         98  8.346582  9.787162

Toys
   percentil     valor       max
0         25  4.354527  7.361547
1         50  4.807866  7.361547
2         75  5.209759  7.361547
3         90  5.560874  7.361547
4         95  5.765912  7.361547
5         98  5.986515  7.361547

Groceries
   percentil     valor       

In [56]:
import numpy as np
import pandas as pd
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor, XGBClassifier

separador = "category"
bins = ventas[separador].unique()
TARGET = ["total_sales"]
FEATURES = [
    'subgroup_cod', 'total_sales_lag_1', 'year',
    'anual_sin_1', 'anual_cos_1', 'mensual_sin_1', 'mensual_sin_2',
    'mean_price_rolling_mean_7d', 'mean_price_rolling_sum_14d',
    'total_sales_rolling_mean_7d', 'total_sales_rolling_sum_7d',
    'total_sales_rolling_mean_14d', 'total_sales_rolling_sum_14d'
]

for bucket in bins:
    print(f"\n=== Bucket: {bucket} ===")
    
    data_bucket = ventas[ventas[separador] == bucket].copy()
    
    # Crear test: última fila por grupo
    test = data_bucket.groupby(['store_cod', 'subgroup_cod']).tail(1)
    train = data_bucket.drop(test.index)
    
    X_train = train[FEATURES]
    y_train = train[TARGET[0]]
    
    X_test = test[FEATURES]
    y_test = test[TARGET[0]]
    
    # 1️⃣ Calcular umbral alto usando solo el train
    umbral_alto = np.percentile(y_train, 95)
    
    # Variable binaria para clasificación
    y_train_cls = (y_train > umbral_alto).astype(int)
    
    scale = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

    clf = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=5,
        colsample_bytree=0.8,
        subsample=0.8,
        scale_pos_weight=scale,
        eval_metric="logloss",
        random_state=42
    )
    clf.fit(X_train, y_train_cls)
    
    # 3️⃣ Modelos de regresión
    # Bajos / medios
    mask_medios = y_train_cls == 0
    reg_med = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        colsample_bytree=0.8,
        reg_lambda=10,
        reg_alpha=0,
        random_state=42,
        n_jobs=-1
    )
    reg_med.fit(X_train[mask_medios], np.log1p(y_train[mask_medios]))
    
    # Altos
    mask_altos = y_train_cls == 1
    reg_alt = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        colsample_bytree=0.8,
        reg_lambda=10,
        reg_alpha=0,
        random_state=42,
        n_jobs=-1
    )
    reg_alt.fit(X_train[mask_altos], np.log1p(y_train[mask_altos]))
    
    # 4️⃣ Predicción combinada
    pred_cls = clf.predict(X_test)  # 0 = medio, 1 = alto
    pred_final = np.zeros(len(X_test))
    
    idx_med = pred_cls == 0
    idx_alt = pred_cls == 1
    
    if idx_med.any():
        pred_final[idx_med] = np.expm1(reg_med.predict(X_test[idx_med]))
    if idx_alt.any():
        pred_final[idx_alt] = np.expm1(reg_alt.predict(X_test[idx_alt]))
    
    # 5️⃣ Métrica
    rmse = np.sqrt(mean_squared_error(y_test, pred_final))
    print(f"Umbral alto (p95): {umbral_alto:.2f}")
    print(f"RMSE bucket {bucket}: {rmse:.4f}")


=== Bucket: Clothing ===


ZeroDivisionError: division by zero

In [55]:
from sklearn.metrics import confusion_matrix
pred_cls = clf.predict(X_test)
y_test_cls = (y_test > umbral_alto).astype(int)
# Matriz de confusión
cm = confusion_matrix(y_test_cls, pred_cls)
print(cm)

[[1182    0]
 [  34    0]]
